In [1]:
import numpy as np
from scipy.optimize import brentq
from scipy.interpolate import interp1d
from sklearn.metrics import roc_curve
from tqdm import tqdm

In [2]:
def EER(labels, scores):
    """
    Computes EER (and threshold at which EER occurs) given a list of (gold standard) True/False labels
    and the estimated similarity scores by the verification system (larger values indicates more similar)
    Sources: https://yangcha.github.io/EER-ROC/ & https://stackoverflow.com/a/49555212/1493011
    """
    fpr, tpr, thresholds = roc_curve(labels, scores, pos_label=True)
    eer = brentq(lambda x: 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    thresh = interp1d(fpr, thresholds)(eer)
    return eer * 100

## For VCTK

In [3]:
def generate_trials(npy_dict, N = 4):
    # generate N positive trials and N negative trals
    labels = []

    scores = []
    keys = list(npy_dict.keys())
    num_keys = len(keys)
    for key in tqdm(keys):
        index = keys.index(key)
        for sample in npy_dict[key]:
            s_index = npy_dict[key].index(sample)
            temp_list = npy_dict[key][0:s_index] + npy_dict[key][s_index + 1:] if s_index < len(npy_dict[key]) - 1 else npy_dict[key][0:s_index]
            embed1 = np.load(sample)
            embed1 = embed1/np.math.sqrt(sum(np.power(embed1,2)))
            for i in range(N):
                labels.append(True)
                compare_index = np.random.randint(0, len(temp_list))
                compare_npy = temp_list[compare_index]
                embed2 = np.load(compare_npy)
                embed2 = embed2/np.math.sqrt(sum(np.power(embed2,2)))
                scores.append(embed1.dot(embed2.T))

                
            for i in range(N):
                labels.append(False)
                temp_klist = keys[0:index] + keys[index + 1:] if index < num_keys - 1 else keys[0:index]
                cmp_key = temp_klist[np.random.randint(0, len(temp_klist))]
                cmp_index = np.random.randint(0, len(npy_dict[cmp_key]))
                embed2 = np.load(npy_dict[cmp_key][cmp_index])
                embed2 = embed2/np.math.sqrt(sum(np.power(embed2,2)))
                scores.append(embed1.dot(embed2.T))

    assert len(scores) == len(labels)
    return labels, scores

In [4]:
def get_dict(npy_list):
    npy_dict = dict()
    for npy in npy_list:
        key = npy.split('/')[-1].split('_')[0].replace('embed-', '')
        if key not in npy_dict.keys():
            npy_dict[key] = []
        npy_dict[key].append(npy)
    return npy_dict

## Speaker EER of the VCTK training set

In [5]:
embed_dir = '../datasets/vctk/synthesizer/embeds/'

pairs = []
with open('../datasets/vctk/synthesizer/train.txt') as testfile:
    for line in testfile.readlines():
        items = line.strip().split('|')
        pairs.append((embed_dir + items[2]))
npy_dict = get_dict(pairs)
labels = [k for k in npy_dict.keys()]

labels_tf, scores = generate_trials(npy_dict, N=100)
EER(labels_tf, scores)

100%|██████████| 100/100 [25:18<00:00, 15.18s/it]


2.0602592097735712